In [5]:
import numpy as np
import robustsp as rsp
from robustsp.DependentData.Auxiliary.helper import *

def ar_est_bip_tau(xxx, P):
    x = np.array(xxx)
    N = len(x) # length of the observation vector
    phi_grid = np.arange(-.99,.991,.05) # coarse grid search
    fine_grid= np.arange(-.99,.991,.001) # finer grid via polynomial interpolation

    kap2 = 0.8724286
    phi_grid = np.arange(-.99,.991,.05) # coarse grid search
    fine_grid= np.arange(-.99,.991,.001) # finer grid via polynomial interpolation
    a_bip_sc = np.zeros([len(phi_grid)]) # residual scale for BIP-AR on finer grid 
    a_sc = np.zeros(len(phi_grid)) # residual scale for AR on finer grid

    # The following was introduced so as not to predict based
    # on highly contaminated data in the
    # first few samples.

    x_tran = x[:min(10,int(np.floor(N/2)))]
    sig_x_tran = np.median(np.abs(x-np.median(x)))

    x_tran[np.abs(x_tran)>3*sig_x_tran] = 3*sig_x_tran*np.sign(x_tran[np.abs(x_tran)>3*sig_x_tran])
    x[1:min(10,int(np.floor(N/2)))] = x_tran[1:min(10,int(np.floor(N/2)))]
    
    if P==0:
        return [], rsp.tau_scale(x)
    elif P==1:
        x_filt, phi_hat, a_scale_final = rsp.bip_ar1_tau(x,N,phi_grid,fine_grid,kap2,P)
    elif P>1:
        phi_hat = np.zeros((P,P))
        x_filt, phi_hat[0,0], a_scale_final = rsp.bip_ar1_tau(x,N,phi_grid,fine_grid,kap2,P)

        npa = lambda x: np.array(x)

        for p in range(1,P):
            for mm in range(len(phi_grid)):

                for pp in range(p):
                    phi_hat[p,pp] =\
                    phi_hat[p-1,pp]-phi_grid[mm]*phi_hat[p-1,p-pp-1]

                predictor_coeffs =\
                np.array([*phi_hat[p,:p], phi_grid[mm]])

                M = len(predictor_coeffs)

                if np.mean(np.abs(np.roots([1, *predictor_coeffs]))<1)==1:
                    lambd = rsp.ma_infinity(predictor_coeffs, 0, 100)
                    sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2))
                else:
                    sigma_hat = 1.483*np.median(np.abs(x-np.median(x)))

                a = np.zeros(len(x))
                a2= np.zeros(len(x))

                for ii in range(p+1,N):

                    xArr = x[ii-1::-1] if ii-M-1 <0 else x[ii-1:ii-M-1:-1]
                    aArr = a[ii-1::-1] if ii-M-1 <0 else a[ii-1:ii-M-1:-1]

                    a[ii] = compA(x[ii],predictor_coeffs,xArr,aArr,sigma_hat)

                    a2[ii] = x[ii] - predictor_coeffs@xArr
                a_bip_sc[mm] = rsp.tau_scale(a[p+1:]) # residual scale for BIP-AR
                a_sc[mm] = rsp.tau_scale(a2[p+1:]) # residual scale for AR

            # tau-estimate under the BIP-AR(p) and AR(p)
            phi, phi2, temp, temp2, ind_max, ind_max2 = tauEstim(phi_grid, a_bip_sc, fine_grid, a_sc)

            # final estimate minimizes the residual scale of the two
            if temp2<temp:
                ind_max=ind_max2
                temp=temp2

            for pp in range(p):
                phi_hat[p,pp] = phi_hat[p-1,pp]-fine_grid[ind_max]*phi_hat[p-1,p-pp-1]

            phi_hat[p,p] = fine_grid[ind_max]

            # final AR(p) tau-scale-estimate depending on phi_hat(p,p)
            if np.mean(np.abs(np.roots([1,*phi_hat[p,:]]))<1) == 1:
                lambd = rsp.ma_infinity(phi_hat[p,:], 0, 100)
                #sigma used for bip-model
                sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2))
            else:
                sigma_hat = 1.483*np.median(np.abs(x-np.median(x)))

            x_filt = np.zeros(len(x))

            for ii in range(p+1,N):
                xArr = x[ii-1::-1] if ii-M-1 <0 else x[ii-1:ii-M-1:-1]
                aArr = a[ii-1::-1] if ii-M-1 <0 else a[ii-1:ii-M-1:-1]

                a[ii] = x[ii]-phi_hat[p,:p+1]@(xArr-aArr\
                +sigma_hat*rsp.eta(aArr/sigma_hat))

                a2[ii]=x[ii]-phi_hat[p,:p+1]@xArr

            if temp2>temp:
                a_scale_final[p+1] = rsp.tau_scale(a[p+1:])
            else:
                a_scale_final[p+1] = rsp.tau_scale(a2[p+1:])

        phi_hat = phi_hat[p,:] # BIP-AR(P) tau-estimate        

        for ii in range(p+1,N):
            x_filt[ii] = x[ii] - a[ii] + sigma_hat*rsp.eta(a[ii]/sigma_hat)

    return phi_hat, x_filt, a_scale_final

# stimmt: ar_est_bip_tau([1,2,3,4],3)
# stimmt: ar_est_bip_tau([1,2,3,4],1)

(0.9900000000000018,
 array([0.        , 1.9975795 , 2.99287637, 3.98400997]),
 array([3.06744407, 1.14251025]))

In [1]:
import robustsp as rsp

rsp.ar_est_bip_tau([1,2,3,4],1)

(0.9900000000000018,
 array([0.        , 1.9975795 , 2.99287637, 3.98400997]),
 array([3.06744407, 1.14251025]))

In [1]:
import numpy as np
import robustsp as rsp

def tau_scale(x):
    b = 0.398545548533895 # E(muler_rho2) under the standard normal distribution
    sigma_m = rsp.m_scale(x)
    return np.sqrt(sigma_m**2 /len(x)  *1/b * np.sum(rsp.muler_rho2(x/sigma_m)))

tau_scale([1,2,3,4,5])

3.714859247303194

In [1]:
import numpy as np
import robustsp as rsp

def bip_ar1_tau(x,N,phi_grid,fine_grid,kap2, P):
    a_scale_final = np.zeros(P+1)
    a_scale_final[0] = rsp.tau_scale(x) # AR(0): residual scale equals observation scale
    
    # grid search for partial autocorrelations
    a_bip_sc = np.zeros(len(phi_grid))
    a_sc = np.zeros(len(phi_grid))
    for mm in range(len(phi_grid)):
        a = np.zeros(len(x)) # residuals for BIP-AR
        a2= np.zeros(len(x)) # residuals for AR
        
        lambd = rsp.ma_infinity(phi_grid[mm], 0, 100)
        sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2)) # sigma used for BIP-model
        for ii in range(1,N):
            a[ii] = x[ii]-phi_grid[mm]*\
            (x[ii-1]-a[ii-1]+sigma_hat*rsp.eta(a[ii-1]/sigma_hat)) # residuals for BIP-AR
            a2[ii] = x[ii] - phi_grid[mm]*x[ii-1] # residuals for AR
             
        a_bip_sc[mm] = rsp.tau_scale(a[1:]) # tau-scale of residuals for BIP-AR
        a_sc[mm] = rsp.tau_scale(a2[1:]) # tau-scale of residuals for AR
    poly_approx = np.polyfit(phi_grid, a_bip_sc, 5) # polynomial approximation of tau scale objective function for BIP-AR(1) tau-estimates
    a_interp_scale = np.polyval(poly_approx,fine_grid) # interpolation of tau scale objective function for BIP-AR(1) tau-estimates to fine grid
    poly_approx2 = np.polyfit(phi_grid,a_sc,5) # polynomial approximation of  tau scale objective function for AR(1) tau-estimates
    a_interp_scale2 = np.polyval(poly_approx2, fine_grid)
    
    temp = np.min(a_interp_scale)
    ind_max = np.argmin(a_interp_scale)
    phi = fine_grid[ind_max] # tau-estimate unter the BIP-AR(1)
    temp2 = np.min(a_interp_scale2)
    ind_max2 = np.argmin(a_interp_scale2)
    phi2 = fine_grid[ind_max2] # tau-estimate under the AR(1)
    
    # final estimate maximizes robust likelihood of the two
    if temp2<temp:
        phi_s = phi2
        temp = temp2
    else:
        phi_s = phi
        
    phi_hat = phi_s # final BIP-tau-estimate for AR(1)
    
    # final AR(1) tau-scale-estimate depending on phi_hat
    lambd = rsp.ma_infinity(phi_hat, 0, 100)
    sigma_hat = a_scale_final[0]/np.sqrt(1+kap2*np.sum(lambd**2))
    a = np.zeros(len(x)) # residuals for BIP-AR
    a2= np.zeros(len(x)) # residuals for AR
    
    x_filt = np.zeros(len(x))
    
    for ii in range(1,N):
        a[ii] = x[ii]-phi_hat*(x[ii-1]-a[ii-1]+sigma_hat*rsp.eta(a[ii-1]/sigma_hat))
        a2[ii] = x[ii] - phi_hat*x[ii-1]
        x_filt[ii] = x[ii] - a[ii] + sigma_hat*rsp.eta(a[ii]/sigma_hat)
        
    if temp2<temp:
        a_scale_final[1] =  rsp.tau_scale(a[1:])
    else:
        a_scale_final[1] = rsp.tau_scale(a2[1:])
        
    return x_filt, phi_hat, a_scale_final

phi_grid = np.arange(-.99,.991,.05) # coarse grid search
fine_grid= np.arange(-.99,.991,.001) # finer grid via polynomial interpolation
rsp.bip_ar1_tau([1,2,3],3,phi_grid,fine_grid,0.8724286,4)

(array([0.        , 1.44722497, 1.43275272]),
 0.9900000000000018,
 array([2.41963251, 1.13688708, 0.        , 0.        , 0.        ]))